# Evaluate syntactic accuracy and semantic accuracy

 contents
- load data 
- Model
    - skipgram
    - skipgram(neg)
    - cbow
    - Glove
- syntactic accuracy and semantic accuracy Result
- correlation

    

# Load data

In [1]:
# load text line by line
with open("questions-words.txt") as f:
    contents = f.read().splitlines() 

In [2]:
# convert all data in question.txt to dictionary

group_contents = []
dict_contents = {}
for line in contents:
    if ':' in line:
        curent_content = line.replace(':',"").strip()
        dict_contents[curent_content]=[]
    else:
        dict_contents[curent_content].append(line.split(" "))

for key in dict_contents.keys():
    print(key,len(dict_contents[key]))

capital-common-countries 506
capital-world 4524
currency 866
city-in-state 2467
family 506
gram1-adjective-to-adverb 992
gram2-opposite 812
gram3-comparative 1332
gram4-superlative 1122
gram5-present-participle 1056
gram6-nationality-adjective 1599
gram7-past-tense 1560
gram8-plural 1332
gram9-plural-verbs 870


In [3]:
# # pick all data
# analogy = {"syntactic":[],"semantic":[]}
# #syntactic accuracy and semantic 
# for key in dict_contents.keys():
#     if "gram" in key:
#         analogy["syntactic"].extend(dict_contents[key])
#     else:
#         analogy["semantic"].extend(dict_contents[key]) 

# for key in analogy.keys():
#     print(key,len(analogy[key]))

In [4]:
# pick some data
analogy = {"syntactic":[],"semantic":[]}
analogy["syntactic"] = dict_contents["gram1-adjective-to-adverb"] + dict_contents["gram7-past-tense"]
analogy["semantic"] = dict_contents["family"] + dict_contents["city-in-state"]
for key in analogy.keys():
    print(key,len(analogy[key]))

syntactic 2552
semantic 2973


In [5]:
# add more data 
import pandas as pd

# Read the CSV file which contain spotify song lyric 
df = pd.read_csv("C:\\Users\\ASUS\\My_Journal\\Text\\My-NLP\\spotify_millsongdata.csv")

# Randomly select 10 song
sample = df.sample(10)

In [6]:
import spacy

corpus = sample["text"]
#load 
nlp = spacy.load("en_core_web_sm")
# reduce space , lower all character and use spacy to tokenize
sparcy_tokenized = [nlp((' '.join(lyric.split())).lower()) for lyric in corpus]
# convert scapy token to str
corpus_tokenized = [[str(word) for word in sublist] for sublist in sparcy_tokenized]


In [7]:
print(len(corpus_tokenized))
# combine all
corpus_tokenized = corpus_tokenized+analogy["semantic"]+analogy["syntactic"]
len(corpus_tokenized)
print(len(corpus_tokenized))

10
5535


In [8]:
# use all question data
# corpus_tokenized =[]

# for key in dict_contents.keys():
#     corpus_tokenized+=dict_contents[key]

In [9]:
# # use some data of question-word
# corpus_tokenized = dict_contents["family"]+dict_contents["gram1-adjective-to-adverb"]

In [10]:
#we want to flatten this (basically merge all list)
def flatten(l):
    return [item for sublist in l for item in sublist]

In [11]:
#2. numericalize



def numericalize_str(corpus_tokenized):

    #2.1 get all the unique words

    vocabs  = list(set(flatten(corpus_tokenized)))  #vocabs is a term defining all unique words your system know

    #2.2 assign id to all these vocabs
    word2index = {v: idx for idx, v in enumerate(vocabs)}

    #add <UNK>, which is a very normal token exists in the world
    vocabs.append('<UNK>') #chaky, can it be ##UNK, or UNKKKKKK, or anything

    #now we have a way to know what is the id of <UNK>
    word2index['<UNK>'] = len(word2index)  #usually <UNK> is 0

    #create index2word dictionary
    #2 min    
    index2word = {v:k for k, v in word2index.items()}

    return vocabs,word2index,index2word



In [12]:
vocabs,word2index,index2word = numericalize_str(corpus_tokenized) 

# Function

In [13]:
import ast
import inspect

def check_local_variables(func):
    # parse the function's code
    source_lines, _ = inspect.getsourcelines(func)
    source_code = '\n'.join(source_lines)
    tree = ast.parse(source_code)
    # keep track of the local variables
    local_vars = set(func.__code__.co_varnames)
    # traverse the AST
    for node in ast.walk(tree):
        if isinstance(node, ast.Name) and node.id not in local_vars:
            print(f"Variable {node.id} is not local.")



In [14]:
import torch.nn as nn
import numpy as np
import torch
import torch.optim as optim
import matplotlib.pyplot as plt

In [15]:
def prepare_sequence(seq, word2index):
    #map(function, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)


In [16]:
#count all the occurrences of vocabs
from collections import Counter

def unigram(corpus_tokenized,vocabs,z = 0.001):
    # Unigram
    
    word_count = Counter(flatten(corpus_tokenized))
    # word_count
    num_total_words = sum([c for w, c in word_count.items()])
    print("total word : ",num_total_words)

    unigram_table = []

    for v in vocabs:
        uw = word_count[v]/num_total_words
        uw_alpha = uw ** 0.75
        uw_alpha_dividebyz = int(uw_alpha / z)
        # print("vocab: ", v)
        # print("distribution: ", uw_alpha_dividebyz)
        unigram_table.extend([v] * uw_alpha_dividebyz)
    
    return unigram_table


In [17]:
import random
#you don't want to pick samples = targets, basically negative samples
#k = number of negative samples - how many? they found 10 is the best
#will be run during training
#after random_batch, 
def negative_sampling(targets, unigram_table, k):
    #targets is already in id.....
    #but the unigram_table is in word....
    #1. get the batch size of this targets
    batch_size = targets.shape[0]
    neg_samples = []
    #2. for each batch
    for i in range(batch_size):
        #randomly pick k negative words from unigram_table
        target_index = targets[i].item()  #looping each of the batch....
        nsample = []
        while len(nsample) < k:
            neg = random.choice(unigram_table)
            #if this word == target, skip this word
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        #append this word to some list
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))  #tensor[], tensor[]
    return torch.cat(neg_samples)  #tensor[[], []]

In [18]:
def pair_data(corpus_tokenized,mode,window_size=2):

    pairs = []

    #for each corpus
    for sent in corpus_tokenized:
        #for each sent ["apple", "banana", "fruit"]
        # # start from window_size end at window_size before last
        for i in range(window_size,len(sent)-window_size):
            center_word = word2index[sent[i]]
            # outside words 
            outside_words = [word2index[sent[j]] for j in range(max(0, i - window_size), min(len(sent), i + window_size + 1)) if j != i]
            for o in outside_words:
                if mode == "skipgram":
                    # append outside word as input center word as output
                    pairs.append([center_word,o])
                elif mode == "cbow":
                    pairs.append([o,center_word])
    
    return pairs

In [19]:
def random_batch(batch_size, pair, window_size=2):
                 
    #only get a batch, not the entire list
    random_index = np.random.choice(range(len(pair)), batch_size, replace=False)
             
    #appending some list of inputs and labels
    random_inputs, random_labels = [], []   
    for index in random_index:
        random_inputs.append([pair[index][0]])  #outside words, this will be a shape of (1, ) --> (1, 1) for modeling
        random_labels.append([pair[index][1]])
        
    return np.array(random_inputs), np.array(random_labels)

In [20]:
def weighting(w_i, w_j, X_ik):   #why we need w_i and w_j, because we can try its co-occurrences, if it's too big, we scale it down
    
    #check whether the co-occurrences between these two word exists???
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1  #why one, so that the probability thingy won't break...(label smoothing)
        
    #maximum co-occurrences; we follow the paper
    x_max = 100
    alpha = 0.75
    
    #if the co-occurrences does not exceed x_max, scale it down based on some alpha
    if x_ij < x_max:
        result = (x_ij/x_max) ** alpha
    else:
        result = 1 #this is the maximum probability you can have
        
    return result

In [21]:
from itertools import combinations_with_replacement

def wdic_co(vocabs,corpus_tokenized,window_size = 2): 

    skip_grams = []
    #for each corpus
    for sent in corpus_tokenized:
        #for each sent ["apple", "banana", "fruit"]
        # # start from window_size end at window_size before last
        for i in range(window_size,len(sent)-window_size):
            center_word = sent[i]
            # outside words 
            outside_words = [sent[j] for j in range(max(0, i - window_size), min(len(sent), i + window_size + 1)) if j != i]
            for o in outside_words:
                # append outside word as input center word as output
                skip_grams.append((center_word,o))

    X_ik_skipgram = Counter(skip_grams)

    X_ik = {} #for keeping the co-occurrences
    weighting_dic = {} #for keeping all the probability after passing through the weighting function

    for bigram in combinations_with_replacement(vocabs, 2):  #we need to also think its reverse
        #if this bigram exists in X_ik_skipgrams
        #we gonna add this to our co-occurence matrix
        if X_ik_skipgram.get(bigram) is not None:
            cooc = X_ik_skipgram[bigram]  #get the co-occurrence
            X_ik[bigram] = cooc + 1 #this is again basically label smoothing....(stability issues (especially when divide something))
            X_ik[(bigram[1], bigram[0])] = cooc + 1  #trick to get all pairs

        
        #apply the weighting function using this co-occurrence matrix thingy    
        weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
        weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)
    
    return weighting_dic,X_ik,skip_grams

In [22]:
import math

def random_batch_glove(batch_size, skip_grams, X_ik, weighting_dic):
    
    #loop through this skipgram, and change it id  because when sending model, it must number
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    #randomly pick "batch_size" indexes
    number_of_choices = len(skip_grams_id)
    random_index = np.random.choice(number_of_choices, batch_size, replace=False) #no repeating indexes among these random indexes
    
    random_inputs = [] #xi, wi (in batches)
    random_labels = [] #xj, wj (in batches)
    random_coocs  = [] #Xij (in batches)
    random_weighting = [] #f(Xij) (in batches)
    #for each of the sample in these indexes
    for i in random_index:
        random_inputs.append([skip_grams_id[i][0]]) #same reason why i put bracket here....
        random_labels.append([skip_grams_id[i][1]])
        
        #get cooc
        #first check whether it exists...
        pair = skip_grams[i]  #e.g., ('banana', 'fruit)
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1 #label smoothing
            
        random_coocs.append([math.log(cooc)])  #1. why log, #2, why bracket -> size ==> (, 1)  #my neural network expects (, 1)
        
        #get weighting
        weighting = weighting_dic[pair]  #why not use try....maybe it does not exist....
        random_weighting.append(weighting)

        
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weighting)
    

# Models

## Skip-gram

In [23]:
class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center_word  = nn.Embedding(voc_size, emb_size)  #is a lookup table mapping all ids in voc_size, into some vector of size emb_size
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center_word, outside_word, all_vocabs):
        #center_word, outside_word: (batch_size, 1)
        #all_vocabs: (batch_size, voc_size)
        
        #convert them into embedding
        center_word_embed  = self.embedding_center_word(center_word)     #(batch_size, 1, emb_size)
        outside_word_embed = self.embedding_outside_word(outside_word)   #(batch_size, 1, emb_size)
        all_vocabs_embed   = self.embedding_outside_word(all_vocabs)     #(batch_size, voc_size, emb_size)
        
        #bmm is basically @ or .dot , but across batches (i.e., ignore the batch dimension)
        top_term = outside_word_embed.bmm(center_word_embed.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) ===> (batch_size, 1)
        
        top_term_exp = torch.exp(top_term)  #exp(uo vc)
        #(batch_size, 1)
        
        lower_term = all_vocabs_embed.bmm(center_word_embed.transpose(1, 2)).squeeze(2)
         #(batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size)
         
        lower_term_sum = torch.sum(torch.exp(lower_term), 1) #sum exp(uw vc)
        #(batch_size, 1)
        
        loss_fn = -torch.mean(torch.log(top_term_exp / lower_term_sum))
        #(batch_size, 1) / (batch_size, 1) ==mean==> scalar
        
        return loss_fn

## Skipgram Neg

In [24]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center_word  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
        self.logsigmoid = nn.LogSigmoid()
        
    def forward(self, center_words, outside_words, negative_words):
        #center_words, outside_words: (batch_size, 1)
        #negative_words:  (batch_size, k)
        
        center_embed  = self.embedding_center_word(center_words)    #(batch_size, 1, emb_size)
        outside_embed = self.embedding_outside_word(outside_words)  #(batch_size, 1, emb_size)
        neg_embed     = self.embedding_outside_word(negative_words) #(batch_size, k, emb_size)
        
        uovc          =  outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2)  #(batch_size, 1)
        ukvc          = -neg_embed.bmm(center_embed.transpose(1, 2)).squeeze(2)  #(batch_size, k)
        ukvc_sum      =  torch.sum(ukvc, 1).view(-1, 1) #(batch_size, 1)
        
        loss = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)  #(batch_size, 1) + (batch_size, 1)
                
        return -torch.mean(loss)  #scalar, loss should be scalar, to call backward()


## CBOW
same as skipgram

## Glove

In [25]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,embed_size):
        super(GloVe,self).__init__()
        self.embedding_center_word = nn.Embedding(vocab_size, embed_size) # center embedding
        self.embedding_outside_word = nn.Embedding(vocab_size, embed_size) # out embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
    def forward(self, center_words, target_words, coocs, weighting):
        center_embeds = self.embedding_center_word(center_words) # [batch_size, 1, emb_size]
        target_embeds = self.embedding_outside_word(target_words) # [batch_size, 1, emb_size]
        
        center_bias = self.v_bias(center_words).squeeze(1)
        target_bias = self.u_bias(target_words).squeeze(1)
        
        inner_product = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]
        
        #note that coocs already got log
        loss = weighting*torch.pow(inner_product +center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)

# Training

params

In [26]:
window_size = 1
unigram_table = unigram(corpus_tokenized,vocabs)
skipgram_data = pair_data(corpus_tokenized,mode = "skipgram",window_size = window_size)
cbow_data = pair_data(corpus_tokenized,mode = "cbow",window_size = window_size)
voc_size   = len(vocabs)
batch_size = 8 
emb_size   = 20
num_neg = 10
learning_rate = 0.0001
all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
weighting_dic,X_ik,skip_gram_word = wdic_co(vocabs,corpus_tokenized,window_size)
num_epochs = 20000

total word :  24545


model

In [27]:
cbow_model = Skipgram(voc_size, emb_size)
skip_gram_model = Skipgram(voc_size, emb_size)
skip_gram_neg_model = SkipgramNeg(voc_size, emb_size) 
glove_model = GloVe(voc_size, emb_size)

optim

In [28]:
cbow_optimizer = optim.Adam(cbow_model.parameters(), lr=learning_rate)
Skipgram_optimizer  = optim.Adam(skip_gram_model.parameters(), lr=learning_rate)
SkipgramNeg_optimizer  = optim.Adam(skip_gram_neg_model.parameters(), lr=learning_rate)
glove_optimizer = optim.Adam(glove_model.parameters(), lr=learning_rate)

trianing loop

skip gram

In [29]:
import time


start_time = time.time()
pre_time = start_time

#for epoch
for epoch in range(num_epochs):

    #get random batch
    input_batch, label_batch = random_batch(batch_size, skipgram_data, window_size)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    
    # print(input_batch.shape, label_batch.shape, all_vocabs.shape)
    
    #loss = model
    loss = skip_gram_model(input_batch, label_batch, all_vocabs)

    #backpropagate
    loss.backward()

    #update alpha
    Skipgram_optimizer.step()
    
    

    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        curr_time = time.time()
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {curr_time-pre_time:.2f} sec")
        pre_time = curr_time

print(f"total time : {curr_time-start_time:.2f} sec")

Epoch 1000 | Loss: 12.452424 | Time: 3.67 sec
Epoch 2000 | Loss: 6.924148 | Time: 3.61 sec
Epoch 3000 | Loss: 6.990582 | Time: 3.48 sec
Epoch 4000 | Loss: 6.338899 | Time: 3.80 sec
Epoch 5000 | Loss: 6.898022 | Time: 3.64 sec
Epoch 6000 | Loss: 5.991551 | Time: 3.54 sec
Epoch 7000 | Loss: 7.471628 | Time: 3.48 sec
Epoch 8000 | Loss: 5.632107 | Time: 3.42 sec
Epoch 9000 | Loss: 5.480076 | Time: 3.52 sec
Epoch 10000 | Loss: 5.551507 | Time: 3.50 sec
Epoch 11000 | Loss: 4.644098 | Time: 3.61 sec
Epoch 12000 | Loss: 2.865330 | Time: 3.52 sec
Epoch 13000 | Loss: 5.892136 | Time: 3.49 sec
Epoch 14000 | Loss: 3.604853 | Time: 3.50 sec
Epoch 15000 | Loss: 3.085553 | Time: 3.51 sec
Epoch 16000 | Loss: 2.797268 | Time: 3.62 sec
Epoch 17000 | Loss: 2.763515 | Time: 3.52 sec
Epoch 18000 | Loss: 5.037763 | Time: 3.51 sec
Epoch 19000 | Loss: 3.985874 | Time: 3.69 sec
Epoch 20000 | Loss: 3.706039 | Time: 3.74 sec
total time : 71.37 sec


cbow

In [30]:
start_time = time.time()
pre_time = start_time

#for epoch
for epoch in range(num_epochs):

    #get random batch
    input_batch, label_batch = random_batch(batch_size, cbow_data, window_size)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    
    # print(input_batch.shape, label_batch.shape, all_vocabs.shape)
    
    #loss = model
    loss = cbow_model(input_batch, label_batch, all_vocabs)

    #backpropagate
    loss.backward()

    #update alpha
    cbow_optimizer.step()
    
    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        curr_time = time.time()
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {curr_time-pre_time:.2f} sec")
        pre_time = curr_time

print(f"total time : {curr_time-start_time:.2f} sec")

Epoch 1000 | Loss: 11.794349 | Time: 3.80 sec
Epoch 2000 | Loss: 7.468336 | Time: 3.77 sec
Epoch 3000 | Loss: 9.400196 | Time: 3.57 sec
Epoch 4000 | Loss: 4.926733 | Time: 3.55 sec
Epoch 5000 | Loss: 5.405241 | Time: 3.73 sec
Epoch 6000 | Loss: 2.546829 | Time: 3.55 sec
Epoch 7000 | Loss: 6.280700 | Time: 3.59 sec
Epoch 8000 | Loss: 2.631036 | Time: 3.49 sec
Epoch 9000 | Loss: 3.455040 | Time: 3.59 sec
Epoch 10000 | Loss: 5.153635 | Time: 3.51 sec
Epoch 11000 | Loss: 3.710968 | Time: 3.65 sec
Epoch 12000 | Loss: 4.874274 | Time: 3.53 sec
Epoch 13000 | Loss: 3.806923 | Time: 3.62 sec
Epoch 14000 | Loss: 5.914106 | Time: 3.61 sec
Epoch 15000 | Loss: 4.010060 | Time: 3.45 sec
Epoch 16000 | Loss: 4.389779 | Time: 3.69 sec
Epoch 17000 | Loss: 4.005140 | Time: 3.75 sec
Epoch 18000 | Loss: 2.982402 | Time: 3.94 sec
Epoch 19000 | Loss: 5.469032 | Time: 3.56 sec
Epoch 20000 | Loss: 4.199884 | Time: 3.78 sec
total time : 72.75 sec


skip gram + negative sampling

In [31]:
#start time
start_time = time.time()
pre_time = start_time
#for epoch
for epoch in range(num_epochs):

    #get random batch
    input_batch, label_batch = random_batch(batch_size, skipgram_data, window_size)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    neg_batch   = negative_sampling(label_batch, unigram_table,num_neg)    
    
    #loss = model
    loss = skip_gram_neg_model(input_batch, label_batch, neg_batch)

    #backpropagate
    loss.backward()
    
    #update alpha
    SkipgramNeg_optimizer.step()
    
    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        curr_time = time.time()
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {curr_time-pre_time:.2f} sec")
        pre_time = curr_time

print(f"total time : {curr_time-start_time:.2f} sec")

Epoch 1000 | Loss: 6.622039 | Time: 3.06 sec
Epoch 2000 | Loss: 5.373026 | Time: 3.21 sec
Epoch 3000 | Loss: 1.705311 | Time: 3.10 sec
Epoch 4000 | Loss: 1.457541 | Time: 3.00 sec
Epoch 5000 | Loss: 3.350996 | Time: 3.02 sec
Epoch 6000 | Loss: 1.857010 | Time: 3.14 sec
Epoch 7000 | Loss: 1.156708 | Time: 2.99 sec
Epoch 8000 | Loss: 1.134656 | Time: 3.10 sec
Epoch 9000 | Loss: 1.027607 | Time: 3.07 sec
Epoch 10000 | Loss: 1.878394 | Time: 3.03 sec
Epoch 11000 | Loss: 0.832130 | Time: 3.08 sec
Epoch 12000 | Loss: 1.782340 | Time: 3.05 sec
Epoch 13000 | Loss: 0.968973 | Time: 3.13 sec
Epoch 14000 | Loss: 3.426190 | Time: 3.02 sec
Epoch 15000 | Loss: 1.864212 | Time: 3.21 sec
Epoch 16000 | Loss: 0.324216 | Time: 3.04 sec
Epoch 17000 | Loss: 1.937943 | Time: 3.01 sec
Epoch 18000 | Loss: 0.817671 | Time: 3.21 sec
Epoch 19000 | Loss: 0.078923 | Time: 3.43 sec
Epoch 20000 | Loss: 0.154513 | Time: 3.29 sec
total time : 62.18 sec


glove

In [32]:

#start time
start_time = time.time()
pre_time = start_time
#for epoch
for epoch in range(num_epochs):

    #get random batch
    input, target, cooc, weightin = random_batch_glove(batch_size,skip_grams = skip_gram_word, X_ik= X_ik,weighting_dic = weighting_dic)
    input_batch    = torch.LongTensor(input)
    target_batch   = torch.LongTensor(target)
    cooc_batch     = torch.FloatTensor(cooc)
    weightin_batch = torch.FloatTensor(weightin)
    
    
    # print(input_batch.shape, label_batch.shape, cooc_batch.shape, weightin_batch)
    
    #loss = model
    loss = glove_model(input_batch, target_batch, cooc_batch, weightin_batch)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    glove_optimizer.step()
    
    #print epoch loss
    if (epoch + 1) % 1000 == 0:
        curr_time = time.time()
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {curr_time-pre_time:.2f} sec")
        pre_time = curr_time

print(f"total time : {curr_time-start_time:.2f} sec")

Epoch 1000 | Loss: 56.563324 | Time: 6.54 sec
Epoch 2000 | Loss: 447.197479 | Time: 5.92 sec
Epoch 3000 | Loss: 150.647552 | Time: 6.04 sec
Epoch 4000 | Loss: 62.777290 | Time: 6.02 sec
Epoch 5000 | Loss: 56.454250 | Time: 6.02 sec
Epoch 6000 | Loss: 115.954620 | Time: 6.01 sec
Epoch 7000 | Loss: 19.194302 | Time: 5.99 sec
Epoch 8000 | Loss: 41.225437 | Time: 6.11 sec
Epoch 9000 | Loss: 54.858322 | Time: 6.05 sec
Epoch 10000 | Loss: 53.335388 | Time: 5.92 sec
Epoch 11000 | Loss: 90.687759 | Time: 6.04 sec
Epoch 12000 | Loss: 46.310501 | Time: 5.90 sec
Epoch 13000 | Loss: 25.042204 | Time: 6.08 sec
Epoch 14000 | Loss: 11.721483 | Time: 5.95 sec
Epoch 15000 | Loss: 67.170059 | Time: 5.99 sec
Epoch 16000 | Loss: 15.251534 | Time: 5.86 sec
Epoch 17000 | Loss: 7.511026 | Time: 6.02 sec
Epoch 18000 | Loss: 72.535904 | Time: 6.02 sec
Epoch 19000 | Loss: 43.044350 | Time: 5.89 sec
Epoch 20000 | Loss: 12.548301 | Time: 5.98 sec
total time : 120.35 sec


# Evaluation

### analogy
from the paper they said "We answer the question “a is to b
as c is to ?” by finding the word d whose representation wd is closest to wb − wa + wc according
to the cosine similaritythey compare word b - word a + word c and find word d by the closest word using cosine similar ". so i just do as they say and count correct answer

In [33]:
def get_embed(word,model):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
    
    word = torch.LongTensor([index])

    center_embed  = model.embedding_center_word(word)
    outside_embed = model.embedding_outside_word(word)
    
    embed = (center_embed + outside_embed) / 2
    
    return  np.array([embed[0][0].item(), embed[0][1].item()])


In [34]:
from numpy import dot
from numpy.linalg import norm

def cos_similarity(a,b):

    cos_sim = dot(a, b)/(norm(a)*norm(b))

    return cos_sim

In [35]:
# this funtion to find most closest word (inspired by my friend i'm not copy)
def similarity_word(wv,vocab_dict):
    closest_idx = np.argmax([cos_similarity(wv, v) for v in vocab_dict.values()])
    return list(vocab_dict.keys())[closest_idx]

In [36]:
# list of models and names
models = [cbow_model,skip_gram_model,skip_gram_neg_model,glove_model]
model_name = ["Cbow","Skip gram","Skip gram Neg","GloVe"]

vocabvec = []
# loop model 
for i in range(len(models)):
    model = models[i]
    syntactic_accuracy = 0
    semantic_accuracy = 0
    vocab_dict = {word: get_embed(word,model) for word in vocabs}
    vocabvec.append(vocab_dict) 

    # loop all question for semantic test
    for q in analogy["semantic"]:
        # Compare embeddings to determine semantic similarity 
        q_embeddings = [get_embed(word, model) for word in q]

        # from paper the answer is word b - word a + word c and find word d by the closest word using cosine similar 
        word_d = q_embeddings[1] - q_embeddings[0] + q_embeddings[2] 

        # see the most similarword in vocabs
        pred_w = similarity_word(word_d,vocab_dict)

        # if predict word is word_d count as correct
        if pred_w == q[3]:
            semantic_accuracy += 1

    semantic_accuracy /= len(analogy["semantic"])
    print(f"{model_name[i]} semantic accuracy : {semantic_accuracy*100:.2f} % " )

    # loop all question for syntactic test
    for q in analogy["syntactic"]:
        q_embeddings = [get_embed(word, model) for word in q]
        # Compare embeddings to determine syntactic similarity (can use cosine similarity)
        word_d = q_embeddings[1] - q_embeddings[0] + q_embeddings[2] 

        pred_w = similarity_word(word_d,vocab_dict)

        # if predict word is word_d count as correct
        if pred_w == q[3]:
            syntactic_accuracy += 1

    syntactic_accuracy /= len(analogy["syntactic"])
    print(f"{model_name[i]} syntactic accuracy : {syntactic_accuracy*100:.2f} % " )

Cbow semantic accuracy : 0.24 % 
Cbow syntactic accuracy : 0.35 % 
Skip gram semantic accuracy : 0.44 % 
Skip gram syntactic accuracy : 0.24 % 
Skip gram Neg semantic accuracy : 0.07 % 
Skip gram Neg syntactic accuracy : 0.08 % 
GloVe semantic accuracy : 0.30 % 
GloVe syntactic accuracy : 0.20 % 


 am i do something wrong lol or i need to use only question.txt to train

# Correration 
in the paper they use spearman methode i guese it is spearmanr from spicy.state 

In [37]:
import pandas as pd
from scipy.stats import spearmanr
# Load the dataset into a DataFrame
data = pd.read_csv("wordsim_similarity_goldstandard.txt", sep='\t', names=["word1", "word2", "human_score"])

# Use the model to predict similarity scores for each word pair
# and store the predicted scores in a new column
# vocab is the list of words in your vocabulary
# list of models and names
models = [cbow_model,skip_gram_model,skip_gram_neg_model,glove_model]
model_name = ["Cbow","Skip gram","Skip gram Neg","GloVe"]

for i in range(4):
    similarities = []
    human_scores = []
    
    # data is the dataframe containing the word pairs and human-assigned similarity scores
    for index,row in data.iterrows():
        word1 = row["word1"]
        word2 = row["word2"]
        human_score = row["human_score"]
        # check if wordsim word appear in vocabs
        if word1 in vocabs and word2 in vocabs:
            # convert the word to its corresponding word embeddings using the trained GloVe model

            center_embeds = get_embed(word1,models[i]) # [batch_size, 1, emb_size]
            target_embeds = get_embed(word2,models[i]) # [batch_size, 1, emb_size]
            #             calculate the cosine similarity between the embeddings
            similarity = cos_similarity(center_embeds,target_embeds)
            similarities.append(similarity)
            human_scores.append(human_score)

    # now you can calculate the correlation between the predicted similarity scores and the human-assigned similarity scores
    correlation = spearmanr(human_scores, similarities)[0]
    print(f"The correlation between the predicted and human similarity of {model_name[i]} is: {correlation:.2f}")




The correlation between the predicted and human similarity of Cbow is: 0.60
The correlation between the predicted and human similarity of Skip gram is: 0.80
The correlation between the predicted and human similarity of Skip gram Neg is: 0.40
The correlation between the predicted and human similarity of GloVe is: 0.20
